# Prompt Engineering: Use OpenAI to Analyze Twitter Data 
This is a simple tutorial teaching prompt engineering basics and analyzing Twitter data with OpenAI large language models (LLM).
Please purchase an [OpenAI API](https://openai.com/index/openai-api/) and store it in a safe place. This tutorial uses [AWS Secretes Manager](https://aws.amazon.com/secrets-manager/) to store the API keys.  

## Large Language Model Basics
LLM repeatable predicts the next world using supervised learning. To predict the following sentence: 

`Learning data science in the cloud with AI`

A model needs to learn to predict the following steps:

|Input|Output|
|:---|---|
|Learning data science |in |
|Learning data science in |the | 
|Learning data science in the |cloud |
|Learning data science in the cloud |with |
|Learning data science in the cloud with |AI|

To train an LLM model:
1. Training a base LLM model on a large amount of training data to predict the next word 
2. Fine-tune on examples where outputs follow instructions in the input 
3. Human rates quality of different LLM outputs 
4. Tune LLM to generate outputs with higher rates using RLHF (Reinforcement learning from human feedback)

## Set up OpenAI Models

Load the API keys with AWS Secrets Manage Function 

In [1]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    return json.loads(secret)

## Install Python libraries.

- pymongo: manage the MongoDB database
- openai: call the OpenAI APIs.

In [2]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 48.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Load the OpenAI API key and define a `openai_help` function.

In [4]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

def openai_help(messages, model=model, temperature =temperature ):
    messages = messages
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature

    )
    return response.choices[0].message.content

Temperature: 
- Low temperature: always choose the most likely response, reliable, predictable responses  
- High temperature: diverse responses, more creative responses

Tokens and Models: 
- LLM predicts tokens, which are commonly occurring sequences of characters. 
- One token is about four characters in English, and 100 tokens are roughly 75 words. Check [token estimate](https://platform.openai.com/tokenizer).
- Different models can process various amounts of tokens at different performance levels and costs. Check [OpenAI models](https://platform.openai.com/docs/models) for more details.

Roles:
- system: specify the overall tone or behavior of the assistant 
- user: instruction given to the LLM
- assistant: LLM responded content, we also can provide content in few-shot promoting or histories of conversations


A simple example using [gtp-4o](https://platform.openai.com/docs/models/gpt-4o) and temperature 0.

In [5]:
messages = [{"role": "user", "content": "What is the capital of USA"}]

print(openai_help(messages))

The capital of the United States is Washington, D.C.


Add a system message asking LLM to act as a high school teacher with different temperatures.

In [6]:
messages = [
    {"role": "system", "content": "use tone as a high school teacher"},
    {"role": "user", "content": "What is the capital of USA"}
    ]

print(openai_help(messages, temperature = 0.8))

The capital of the United States is Washington, D.C. Remember, it's not part of any state and has its own unique status. If you have any more questions about capitals or geography, feel free to ask!


Add assistant messages to teach LLM what `##` is.

In [7]:
messages = [
    {"role": "user", "content": "What is 1##1"},
    {"role": "assistant", "content": "it is 11"},
    {"role": "user", "content": "What is 2##2"},
    {"role": "assistant", "content": "it is 22"},
    {"role": "user", "content": "What is 3##3"},
    ]
print(openai_help(messages))

It is 33.


## Prompt Engineering Principles 
- Use delimiters to separate different parts of a prompt to provide clear instructions and prevent prompt injections.
- Structure outputs in JSON documents or other formats to use the outputs in subsequent steps 
- Few-shot promoting: provide successful examples of a task and then ask the model to perform a similar task. 
- Chain of thought reasoning: request a series of reasoning steps in prompts to help the model achieve correct answers
- Chain of prompts: split a task into multiple prompts where each prompt can focus on a sub-task at a time and take different actions at different stages. It saves tokens, is easier to test, can involve human input, or use external tools.
- Interactive process 
  1. Try something first 
  2. Analyses the result, identify errors, and redefine the prompt 
  3. Test the prompts with different datasets 


An example using delimiters, structured output and few-shot promoting:

In [8]:
delimiter = '###'
sentence1 = 'I love cat.'
sentence2 = 'I love dog.'
messages = [
    {"role": "system", "content": f"""analyze the sentiment in a sentence delimitered by {delimiter},
                                     return the result as a JSON document"""},
    {"role": "user", "content": f"{delimiter}{sentence1}{delimiter}"},
    {"role": "assistant", "content": "{sentiment:positive}"},
    {"role": "user", "content": f"{delimiter}{sentence2}{delimiter}"}
    ]

print(openai_help(messages))

{ "sentiment": "positive" }


## Analyze Twitter data

### Connect to the MongoDB cluster

In [9]:
import pymongo
from pymongo import MongoClient
mongodb_connect = get_secret('mongodb')['connection_string']

mongo_client = MongoClient(mongodb_connect)
db = mongo_client.demo # use or create a database named demo
tweet_collection = db.tweet_collection #use or create a collection named tweet_collection
tweet_collection.create_index([("tweet.id", pymongo.ASCENDING)],unique = True) # make sure the collected tweets are unique

'tweet.id_1'

### Extract Tweets

In [10]:
filter={

    
}
project={
    'tweet.text': 1, 
    'tweet.id': 1
}
#rename the client to mongo_client
result = mongo_client['demo']['tweet_collection'].find(
  filter=filter,
  projection=project
)

In [11]:
tweet_data = []
for tweet in result:
    tweet_data.append(tweet['tweet']['text'])
print (tweet_data)

['siento q la trisha paytas y la aimep3 son lo mismo', 'girl i went to school with just met trisha paytas at the airport ????', 'Trisha Paytas singing I Love you Jesus and then Freaky like 5 minutes later was amazing', 'I saw Trisha Paytas live last night and it was just so iconic I cannot even describe the immaculate vibes https://t.co/9MpHR9SYAK', '@zPopDrug Is that trisha paytas😭😭😭', 'i’m gonna get desperate for a second and say if you saw trisha paytas in connecticut yesterday and got approached by a girl to be in her tiktok, PLEASE dm me. i don’t know her name and i need to know how stupid i look in this video.', 'como pode em um segundo eu estar escutando kate bush e logo depois trisha paytas', '@TotallyTrasha "stalking" and how many of us have her blocked on all platforms lol?? sorry, I just think people who enjoy Trisha Paytas should be aware they\'re supporting racism, unchecked consumerism &amp; disordered eating. Nevermind the child exploitation.', '@njzditto so was Trisha P

In [12]:
print('Number of tweets: ',len(tweet_data))

Number of tweets:  93


### Summarization 
- Analyze election tweets with delimiters 
- Change the size of the summarization 
- Summarize tweets and focus on different perspectives. 

In [13]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter}"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets revolve around Trisha Paytas, a controversial internet personality known for her polarizing presence. Many tweets mention encounters with her, such as meeting her at the airport or attending her live shows, which are described as iconic and full of energy. Some tweets criticize her for past actions, including accusations of racism and other problematic behavior. Others express admiration for her, noting her influence and success despite controversies. There are also mentions of her appearances on platforms like SNL and her involvement in various projects. Overall, the tweets reflect a mix of admiration, criticism, and fascination with Trisha Paytas.


In [14]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    limit the summary to 20 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

Trisha Paytas is a controversial figure, sparking mixed reactions online due to her performances, public appearances, and past actions.


In [15]:
messages = [
    {"role": "system", "content": f"""provide a brief summary of the tweets delimited by {delimiter},
                                    focus on how people discuss AI,
                                    limit the summary to 50 words"""},
    {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter}"},
    ]

print(openai_help(messages))

The tweets focus on Trisha Paytas, with discussions ranging from her public appearances and performances to controversies surrounding her actions and statements. People express mixed opinions, highlighting her influence and polarizing nature, while some compare her to other public figures and discuss her impact on social media culture.


### Moderation 
- Iterate each tweet and use the [moeration endpoint](https://platform.openai.com/docs/api-reference/moderations) to identify flagged tweets
- Print flagged tweets


In [16]:
def flag_help(tweet):
    response = client.moderations.create(
        model="omni-moderation-latest",
        input=tweet)

    if response.results[0].flagged:
        print('===')
        cat_dict = response.results[0].categories.to_dict()
        for cat in cat_dict.keys():
            if cat_dict.get(cat):
                print (cat)
                print(tweet)

In [18]:
for tweet in tweet_data:
    flag_help(tweet)

===
harassment
@TotallyTrasha "stalking" and how many of us have her blocked on all platforms lol?? sorry, I just think people who enjoy Trisha Paytas should be aware they're supporting racism, unchecked consumerism &amp; disordered eating. Nevermind the child exploitation.
===
harassment
@fieldsy69 @NicTNemeth @mandysacs More like Trisha Paytas or Sunny?  The show off has a type...fat.
===
violence
EL LORE DE TRISHA PAYTAS
===
sexual
RT @WtfuckClub: Trisha Paytas Nude/Pussy https://t.co/HVAEoJ5DKY
===
harassment
The way I’ll never like Trisha paytas because she did black face
===
harassment
@itsavibe And not a single trans actress or trans musician has been on this show. Racist, homophobic and transphobic cis straight white people really seem to get a pass, recently, from SNL. Elon hosted once. Shane Gillis hosted twice. Trisha Paytas appeared on the show and now Wallen 🤦‍♀️🤦‍♀️.
===
harassment
RT @KiDDHall: @PopCrave Rachel Zegler didn't give a fuck about  collaborating with  Zionist

### Transforming
- Translating to a different language 
- Transform tones, such as formal vs. informal.  


In [19]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""translate the tweets delimited by {delimiter} into Chinese"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

我觉得 Trisha Paytas 和 Aimep3 是一样的。
我以前上学时的一个女生刚刚在机场遇到了Trisha Paytas？？？
Trisha Paytas在唱完《I Love you Jesus》后五分钟又唱《Freaky》，真是太精彩了
昨晚我看了Trisha Paytas的现场表演，真是太经典了，我甚至无法形容那种完美的氛围。https://t.co/9MpHR9SYAK
@zPopDrug 那是 Trisha Paytas 吗😭😭😭
我有点绝望地说一下，如果你昨天在康涅狄格州看到Trisha Paytas，并且有一个女孩找你拍她的TikTok视频，请给我发私信。我不知道她的名字，我需要知道我在这个视频里看起来有多傻。
怎么可能我一秒钟前还在听凯特·布什，然后就听崔莎·佩塔斯
@TotallyTrasha “跟踪”以及我们中有多少人在所有平台上屏蔽了她，哈哈？抱歉，我只是觉得喜欢Trisha Paytas的人应该意识到他们在支持种族主义、不受控制的消费主义和饮食失调。更不用说对儿童的剥削了。
@njzditto 特里莎·佩塔斯也是如此，但她决定在出道前生孩子 https://t.co/s5YOqfbvBQ
@Brit_Named_Tom 实际上是 Trisha Paytas，但天啊，施虐者是 Jack Dylan Grazer，就是《雷霆沙赞！》里的另一个孩子。
RT @mushiefairie: 我很高兴我不是名人，因为人们故意误解每个人的方式，对事情有意见却不了解全部情况……
@hobit_a @wildatheart1989 @afvulax @reem_ies 还记得几天前泰勒祝埃尔顿·约翰生日快乐吗？还想上Trisha Paytas的节目？哦，等等，那其实是Chappell Roan！
@katieroo_25 @afvulax @reem_ies Chappell不是和Elton John和Trisha Paytas是朋友吗
RT @cubbypowered: 嘿，我的小鱼儿们，今天我想____自己，Trisha Paytas 取消好友的恶搞，LiveLeak 汤在旁边……
Trisha Paytas 转发了我的推文，哈哈哈哈，我可是头号“爸爸问题”和“伤心男孩2005”的粉丝
@BrettGibbs17 我完全不同意并谴责Rachel选择

In [20]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""rewrite the tweets delimited by {delimiter} in the tone like Stewie """},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]

    print(openai_help(messages).strip(delimiter))

I dare say, Trisha Paytas and Aimep3 are practically indistinguishable, aren't they?
Oh, splendid! A former classmate of mine has had the distinct pleasure of encountering the illustrious Trisha Paytas at the airport. How utterly delightful!
Ah, the delightful juxtaposition of Trisha Paytas serenading the heavens with "I Love You Jesus" and then promptly descending into the delightful chaos of "Freaky" mere moments later. Truly, a performance worthy of a standing ovation, or at least a bemused smirk.
Ah, yes, I had the distinct pleasure of witnessing Trisha Paytas in the flesh last night. The experience was nothing short of iconic, my dear simpletons. The vibes were so immaculate, even I, with my superior intellect, find it challenging to articulate. https://t.co/9MpHR9SYAK
Oh, do my eyes deceive me, or is that the illustrious Trisha Paytas gracing us with her presence? How utterly delightful.
Oh, splendid. I'm about to descend into the depths of desperation, but here it goes: If you h

### Inferring
- Use step-by-step instructions with delimiters to:
  1. Identify sentiments
  2. Identify emotions
  3. Extract mentioned people's names
  3. Identify whether a tweet supports Democratic, Republican, or unknown 
  4. Extract outputs into a structured JSON document. 
- Identify topics from Tweets. 


In [21]:
for tweet in tweet_data:
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a single word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    print(openai_help(messages))

{
  "sentiment": "neutral",
  "emotion": "indifferent",
  "mentioned": ["trisha paytas", "aimep3"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "surprise",
  "mentioned": ["trisha paytas"],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "amusement",
  "mentioned": ["Trisha Paytas"],
  "support": "neutral"
}
{
  "sentiment": "positive",
  "emotion": "excitement",
  "mentioned": ["Trisha Paytas"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "curiosity",
  "mentioned": ["trisha paytas"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "desperation",
  "mentioned": ["trisha paytas"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "confusion",
  "mentioned": ["kate bush", "trisha paytas"],
  "support": "neutral"
}
{
  "sentiment": "negative",
  "emotion": "disapproval",
  "mentioned": ["Trisha Paytas"],
  "support": "neutral"
}
{
  "sentiment": "neutral",
  "emotion": "informative",
  "menti

In [22]:

messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} to identify 10 topics, 
                                  Do not wrap the json codes in JSON markers """},
        {"role": "user", "content": f"{delimiter}{tweet_data}{delimiter} "}]
print(openai_help(messages))

{
  "1": "Trisha Paytas' Public Appearances",
  "2": "Controversies Surrounding Trisha Paytas",
  "3": "Trisha Paytas' Music and Performances",
  "4": "Trisha Paytas and Social Media Interactions",
  "5": "Trisha Paytas' Influence and Comparisons",
  "6": "Trisha Paytas and Celebrity Interactions",
  "7": "Trisha Paytas' Personal Life and Family",
  "8": "Trisha Paytas and Online Criticism",
  "9": "Trisha Paytas' Cultural Impact",
  "10": "Trisha Paytas and Media Coverage"
}


### Expanding with multiple prompts 
- Identify which party receives majority supports
- Provide contexts in the system message
- Create a chatbot to answer users’ inquiry  


In [23]:
analysis_result = []
from tqdm import tqdm
for tweet in tqdm(tweet_data):
    messages = [
        {"role": "system", "content": f"""analyze the tweet delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} identify the tweet sentiment in a single word, either positive, negative or neutral;
                                        step 2 {delimiter} identify the emotions expressed in the tweet with a single word;
                                        step 3 {delimiter} extract the mentioned peoples;
                                        step 4 {delimiter} detect whether the tweet support Democratic or Replublican, return the resunt in a singple word;
                                        step 5 {delimiter} organize the result in a json document with the keys <sentiment>, <emontion>,<mentioned>, <support>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{tweet}{delimiter} "}]
    analysis_result.append(openai_help(messages))


100%|██████████| 93/93 [01:29<00:00,  1.04it/s]


In [24]:
print(analysis_result)

['{\n  "sentiment": "neutral",\n  "emotion": "indifference",\n  "mentioned": ["trisha paytas", "aimep3"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "surprise",\n  "mentioned": ["trisha paytas"],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "amusement",\n  "mentioned": ["Trisha Paytas"],\n  "support": "neutral"\n}', '{\n  "sentiment": "positive",\n  "emotion": "excitement",\n  "mentioned": ["Trisha Paytas"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "curiosity",\n  "mentioned": ["trisha paytas"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "desperation",\n  "mentioned": ["trisha paytas"],\n  "support": "neutral"\n}', '{\n  "sentiment": "neutral",\n  "emotion": "confusion",\n  "mentioned": ["kate bush", "trisha paytas"],\n  "support": "neutral"\n}', '{\n  "sentiment": "negative",\n  "emotion": "disapproval",\n  "mentioned": ["Trisha Paytas"],\n  "support": "neutral"\n}'

In [25]:
messages = [
        {"role": "system", "content": f"""analyze the tweet analysis reuslt delimited by {delimiter} in the following steps:
                                        step 1 {delimiter} count the number of tweets that support Democratic and Republican;
                                        step 2 {delimiter} identify the common sentiments and emotoions to each mentioned people;
                                        step 3 {delimiter} organize the result in a json document with keys <Democratic count>, <Republican count>, <people name>
                                         Do not wrap the json codes in JSON markers and only return the json document"""},
        {"role": "user", "content": f"{delimiter}{analysis_result}{delimiter} "}]
analysis_summary = openai_help(messages)
print(analysis_summary)

```json
{
  "Democratic count": 2,
  "Republican count": 0,
  "people name": {
    "Trisha Paytas": {
      "sentiments": {
        "neutral": 54,
        "positive": 10,
        "negative": 10
      },
      "emotions": {
        "indifference": 7,
        "surprise": 5,
        "amusement": 2,
        "excitement": 6,
        "curiosity": 8,
        "desperation": 1,
        "confusion": 2,
        "disapproval": 2,
        "informative": 2,
        "anger": 3,
        "frustration": 5,
        "nostalgia": 5,
        "affection": 1,
        "none": 3,
        "admiration": 3,
        "disgust": 3,
        "disdain": 2,
        "love": 1,
        "inspiration": 1,
        "loneliness": 1,
        "calm": 2,
        "disappointment": 1,
        "flattered": 1
      }
    },
    "mushiefairie": {
      "sentiments": {
        "neutral": 10
      },
      "emotions": {
        "frustration": 8,
        "relief": 2
      }
    },
    "Rachel Zegler": {
      "sentiments": {
        "neut

## Create a chatbot

In [ ]:
from openai import OpenAI

openai_api_key  = get_secret('openai')['api_key']
client = OpenAI(api_key=openai_api_key)
model = 'gpt-4o'
temperature = 0

chat_history = [

{"role": "system", "content": f"""you are a chabot answer user questions based on the tweets,
                                {delimiter}{tweet_data}{delimiter}, 
                                if user mentioned a people name in the {delimiter}{analysis_summary}{delimiter} people field,report the corresponding sentiment and emotion,
                            
                            """}
]

def chatbot(prompt):

    chat_history.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,  # Use the model you prefer
        messages=chat_history
    )

    reply = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": reply})
    
    return reply

In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit']:
        print("Chatbot: Goodbye!")
        break
    reply = chatbot(user_input)
    print(f"Chatbot: {reply}")


## Reference
- Isa Fulford and Andrew Ng. n.d.-a. *“Building Systems with the ChatGPT API.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/building-systems-with-chatgpt/.
- ———. n.d.-b. *“ChatGPT Prompt Engineering for Developers.”* DeepLearning.AI. Accessed October 25, 2024. https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/.
- OpenAI. n.d. *“OpenAI Documents.”* OpenAI. Accessed October 18, 2024. https://platform.openai.com.
